### Specifications
<b>1. Data </b>
- Fuzzy match results w/ columns: filename (which contains proposal release no and comment no), firm, firm ticker, match score
- Formatted final rule data w/ columns: File No,Proposed Rule ID,Proposed Rule Date,Proposed Rule Text,Final Rule ID,Final Rule Date,Final Rule Text
- Merging and cleaning the two files, I get rules_metadata.csv w/ columns: Release no.,Firms_Name,Firms_Ticker,Proposed date,Final date,Number of comments

<b>2. Regression</b>
- Data used: 1) time range: for each rule, use the final rule date to backtrack and forward 5 days. The final rule date and the next 5 days are considered "Post," 2) CRSP daily data for the commenters of the rules and 12 sector ETFs during the period.
- Regression formula: $Vol = \alpha_i, \beta_1 * post + \beta_2 * post * commented + \beta_3 * log(num comment) + \epsilon$


In [13]:
# ==========================================================
# SEC Policy Impact — Simple Vol Regression (Permno-only)
# Final sample: all commenters (from rules_metadata2.csv) + 12 sector ETFs
# Event date: Final date
# Window for regression: tau ∈ {-5,...,+5}
# Vol measure: choose 21-day realized or Parkinson
# ==========================================================

import os
import numpy as np
import pandas as pd
import wrds
import statsmodels.formula.api as smf

# -----------------------
# 0) CONFIG / PARAMETERS
# -----------------------
BASE = "/Users/dorajyl/Desktop/w/UROP/Part 2. Data Cleaning/based_on_comment/data"
RULES_META_CSV = os.path.join(BASE, "rules_metadata2.csv")
SECTOR_PERMNO_CSV = os.path.join(BASE, "sp500_sector_permno.csv")

OUT = os.path.join(BASE, "checkpoints_influential_simple")
os.makedirs(OUT, exist_ok=True)

# Influential filter (choose ONE mode) 
# for sample selection if run time is an issue
FILTER_MODE = "topN"     # "topN" | "percentile" | "mincount"
TOP_N = 75               # used if FILTER_MODE="topN"
PCTL = 0.90              # used if FILTER_MODE="percentile"
MIN_COMMENTS = 50        # used if FILTER_MODE="mincount"

# CRSP pull window (wide) & RV buffer
EVENT_WINDOW = 90        # days pre/post for CRSP pull
RV_BUFFER = 30           # extra days for realized-vol lookback

# Winsorization switch
APPLY_WINSOR = False
WINSOR_P = 0.01          # 1% tails if APPLY_WINSOR=True

CHUNK = 800  # permno chunk size for CRSP VALUES join

# -----------------------
# Helpers 
# -----------------------
def choose_event_date(row):
    """Use Final date only (drop if missing)."""
    if pd.notna(row['Final date']) and str(row['Final date']).strip():
        return pd.to_datetime(row['Final date'])
    return pd.NaT

def split_permnos(s):
    """Split Firms_permno '19583; 53081; ...' into list of ints."""
    if pd.isna(s):
        return []
    pieces = [p.strip() for p in str(s).split(';')]
    out = []
    for p in pieces:
        if not p:
            continue
        try:
            out.append(int(p))
        except ValueError:
            continue
    return out

def parkinson_vol(H, L):
    with np.errstate(all='ignore'):
        rng = np.log(H / L)
        var = (1.0 / (4.0 * np.log(2.0))) * (rng ** 2)
        return np.sqrt(var)

def realized_vol_21(ret):
    r = pd.to_numeric(ret, errors='coerce')
    return r.rolling(21, min_periods=15).std()

def corwin_schultz_spread_series(g):
    Ht, Lt = g['hi_q'], g['lo_q']
    Hm1, Lm1 = Ht.shift(1), Lt.shift(1)
    a_t  = (np.log(Ht / Lt))**2
    a_m1 = (np.log(Hm1 / Lm1))**2
    two  = (np.log(np.maximum(Ht, Hm1) / np.minimum(Lt, Lm1)))**2
    gamma = np.sqrt(np.maximum(0.0, a_t + a_m1 - two))
    S = 2.0 * (np.exp(gamma) - 1.0)
    return S.clip(0, 1)

def winsorize_by_year(df, col, p=0.01):
    def _w(s):
        lo, hi = s.quantile(p), s.quantile(1-p)
        return s.clip(lo, hi)
    return df.groupby(df['date'].dt.year)[col].transform(_w)

def fetch_crsp_slice_for_permnos(db, permnos, start_date, end_date, chunk=800):
    """Server-side filter via a chunked VALUES join for performance."""
    out = []
    cols = ['permno','date','prc','ret','vol','shrout','askhi','bidlo']
    if len(permnos) == 0:
        return pd.DataFrame(columns=cols)
    for i in range(0, len(permnos), chunk):
        chunk_ids = permnos[i:i+chunk]
        values_rows = ",".join(f"({int(p)})" for p in chunk_ids)
        q = f"""
            WITH perm_tmp(permno) AS (VALUES {values_rows})
            SELECT d.permno, d.date, d.prc, d.ret, d.vol, d.shrout, d.askhi, d.bidlo
            FROM crsp.dsf d
            JOIN perm_tmp t ON d.permno = t.permno
            WHERE d.date BETWEEN '{start_date}' AND '{end_date}'
        """
        out.append(db.raw_sql(q))
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame(columns=cols)

# -----------------------
# Step 0: Load metadata
# -----------------------
req = [
    'Release no.','Firms_Name','Firms_Ticker','Firms_permno',
    'Proposed date','Final date','Number of comments'
]
meta = pd.read_csv(RULES_META_CSV)
missing = [c for c in req if c not in meta.columns]
if missing:
    raise ValueError(f"Missing columns in metadata: {missing}")

meta['event_date']   = meta.apply(choose_event_date, axis=1)
meta['n_comments']   = pd.to_numeric(meta['Number of comments'], errors='coerce').fillna(0.0)
meta['log_comments'] = np.log1p(meta['n_comments'])

meta = meta.dropna(subset=['Release no.','event_date']).copy()

# -------------------------------
# Step 0a: Filter "influential"
# -------------------------------
grp_max = meta.groupby('Release no.')['n_comments'].max()
if FILTER_MODE == "topN":
    keep_ids = grp_max.sort_values(ascending=False).head(TOP_N).index
elif FILTER_MODE == "percentile":
    cutoff = grp_max.quantile(PCTL)
    keep_ids = grp_max[grp_max >= cutoff].index
elif FILTER_MODE == "mincount":
    keep_ids = grp_max[grp_max >= MIN_COMMENTS].index
else:
    raise ValueError("FILTER_MODE must be 'topN' or 'percentile' or 'mincount'")

meta_inf = meta[meta['Release no.'].isin(keep_ids)].copy()

# ------------------------------------------------------
# Step 1: Commenter universe (permno from Firms_permno)
# ------------------------------------------------------
comment_rows = []
for _, r in meta_inf.iterrows():
    rel = r['Release no.']
    ed  = r['event_date']
    logc = r['log_comments']
    permnos = split_permnos(r['Firms_permno'])
    for p in permnos:
        comment_rows.append({
            'Release no.': rel,
            'event_date':  ed,
            'log_comments': logc,
            'permno': p
        })

commenters_permno = (
    pd.DataFrame(comment_rows)
    .dropna(subset=['permno'])
    .drop_duplicates()
)
commenters_permno['commented'] = 1

if commenters_permno.empty:
    raise RuntimeError("No commenter PERMNOs after filtering influential rules.")

# ------------------------------------------------------
# Step 2: Sector ETF universe from sector_permno.csv
# ------------------------------------------------------
sector = pd.read_csv(SECTOR_PERMNO_CSV)
if 'permno' not in sector.columns:
    raise ValueError("sector_permno.csv must have a 'permno' column.")
sector['permno'] = pd.to_numeric(sector['permno'], errors='coerce')
sector = sector.dropna(subset=['permno']).drop_duplicates()

etf_rows = []
for _, r in meta_inf[['Release no.','event_date','log_comments']].drop_duplicates().iterrows():
    rel, ed, logc = r['Release no.'], r['event_date'], r['log_comments']
    for p in sector['permno']:
        etf_rows.append({
            'Release no.': rel,
            'event_date':  ed,
            'log_comments': logc,
            'permno': int(p)
        })

etf_permno = pd.DataFrame(etf_rows).drop_duplicates()
etf_permno['commented'] = 0

if etf_permno.empty:
    raise RuntimeError("No sector ETF permnos found; check sector_permno.csv.")

# --------------------------------------------
# Step 3: Exposure = commenters + sector ETFs
# --------------------------------------------
exposure = (
    pd.concat([commenters_permno, etf_permno], ignore_index=True)
    .drop_duplicates()
    .reset_index(drop=True)
)

if exposure.empty:
    raise RuntimeError("Exposure empty; check commenter permno and ETF mapping.")

# -------------------------------------------------------------------
# Step 4: Pull CRSP dsf for permno universe
# -------------------------------------------------------------------
db = wrds.Connection()  # needs WRDS/Duo

permno_universe = sorted(exposure['permno'].unique())
event_min = (meta_inf['event_date'].min() - pd.Timedelta(days=EVENT_WINDOW + RV_BUFFER)).date()
event_max = (meta_inf['event_date'].max() + pd.Timedelta(days=EVENT_WINDOW + RV_BUFFER)).date()

crsp = fetch_crsp_slice_for_permnos(db, permno_universe, event_min, event_max, chunk=CHUNK)

crsp['date'] = pd.to_datetime(crsp['date'])
for c in ['prc','ret','vol','shrout','askhi','bidlo']:
    crsp[c] = pd.to_numeric(crsp[c], errors='coerce')
crsp['prc_abs'] = crsp['prc'].abs()
crsp = crsp.sort_values(['permno','date']).reset_index(drop=True)
crsp = crsp.rename(columns={'askhi':'hi_q','bidlo':'lo_q'})

# ------------------------------------------------------
# Step 5: Volatility proxies + winsorization (optional)
# ------------------------------------------------------
crsp['pk_vol'] = parkinson_vol(crsp['hi_q'], crsp['lo_q'])

# use CRSP 'ret' when available; fallback to pct_change
ret_used = crsp['ret'].copy()
ret_fallback = crsp.groupby('permno')['prc_abs'].pct_change()
ret_used = ret_used.where(ret_used.notna(), ret_fallback)

crsp['rv21'] = (
    ret_used
    .groupby(crsp['permno'])
    .apply(realized_vol_21)
).reset_index(level=0, drop=True)

cs_list = []
for _, g in crsp.groupby('permno', sort=False):
    cs_list.append(corwin_schultz_spread_series(g))
crsp['cs_spread'] = pd.concat(cs_list).sort_index()
crsp['cs_spread_smooth'] = crsp.groupby('permno')['cs_spread'] \
    .transform(lambda s: s.rolling(5, min_periods=3).median())

for col in ['pk_vol','rv21','cs_spread_smooth']:
    if APPLY_WINSOR:
        crsp[col + '_w'] = winsorize_by_year(crsp, col, p=WINSOR_P)
    else:
        crsp[col + '_w'] = crsp[col]

# ----------------------------------------------
# Step 6: Stack event windows 
# ----------------------------------------------
panel = exposure.merge(crsp, on='permno', how='left')
panel['tau']  = (panel['date'] - panel['event_date']).dt.days
panel = panel[(panel['tau'] >= -EVENT_WINDOW) & (panel['tau'] <= EVENT_WINDOW)].copy()


Loading library list...
Done


---------------------------------------------------------------------------

In [18]:
panel.to_csv(os.path.join(OUT, "panel_full.csv"), index=False)
panel.columns

Index(['Release no.', 'event_date', 'log_comments', 'permno', 'commented',
       'date', 'prc', 'ret', 'vol', 'shrout', 'hi_q', 'lo_q', 'prc_abs',
       'pk_vol', 'rv21', 'cs_spread', 'cs_spread_smooth', 'pk_vol_w', 'rv21_w',
       'cs_spread_smooth_w', 'tau'],
      dtype='object')

In [21]:
# ==========================================================
# Step 7: Simple regression on τ ∈ {-5,...,+5}
#     Vol_{i,t} = α_i + β1*post + β2*post*commented + β3*log_comments + ε
# ==========================================================
# Tight analysis window for regression
EVENT_WINDOW_A = 5       # tau ∈ [-5, +5]
panel_a = panel[(panel['tau'] >= -EVENT_WINDOW_A) & (panel['tau'] <= EVENT_WINDOW_A)].copy()

# post = 1 if tau >= 0 (final date and above)
panel_a['post'] = (panel_a['tau'] >= 0).astype(int)
# commented = 1 for commenters, 0 for sector ETFs
panel_a['commented'] = panel_a['commented'].astype(int)
# Vol outcome
panel_a['Vol'] = panel_a['rv21']
# log_comments (per rule) — fill if any missing
panel_a['log_comments'] = panel_a['log_comments'].fillna(0)
# rule fixed effects
panel_a['rule_id'] = panel_a['Release no.'].astype(str)
panel_a.dropna(subset=['Vol'], inplace=True)
# Regression model
mod = smf.ols(
    'Vol ~ post + post:commented + log_comments + C(rule_id)',
    data=panel_a
)

# cluster by rule
res = mod.fit(
    cov_type='cluster',
    cov_kwds={'groups': panel_a['rule_id']}
)
with open(os.path.join(OUT, "result_rv21.txt"), "w") as f:
    f.write(str(res.summary()))

print("\n=== Simple Vol Regression (rule FE; clustered by rule) ===")
print(res.summary())



=== Simple Vol Regression (rule FE; clustered by rule) ===
                            OLS Regression Results                            
Dep. Variable:                    Vol   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.468
Method:                 Least Squares   F-statistic:                 2.297e-08
Date:                Tue, 18 Nov 2025   Prob (F-statistic):               1.00
Time:                        09:23:13   Log-Likelihood:                 57428.
No. Observations:               16492   AIC:                        -1.148e+05
Df Residuals:                   16439   BIC:                        -1.143e+05
Df Model:                          52                                         
Covariance Type:              cluster                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------

/Users/dorajyl/Desktop/w/UROP/venv/lib/python3.9/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 53, but rank is 3
  warnings.warn('covariance of constraints does not have full '
